### Q1

In [13]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [14]:
!python -V

Python 3.10.13


In [4]:
# Libraries
import pickle
import pandas as pd
from pathlib import Path

In [5]:
# Set constants
work_path = Path("")
#dv_path = work_path / "dv.json"
output_file = work_path / "output"

In [6]:
#text_file = dv_path.read_text()
#text_dict = json.loads(text_file)

#dv = DictVectorizer()
#x_train = dv.fit_transform(text_dict)

In [7]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [8]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [9]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [10]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [11]:
y_pred.std()

6.247488852238703

#### Q2

In [12]:
year = 2023
month = 3

In [27]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [28]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['prediction'] = y_pred

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [29]:
!du -h output

66M	output
